In [1]:
# !pip3 install malaya-speech -U --no-deps

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import malaya_speech
import json

malaya_speech.__version__

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

'1.2.6.1'

In [4]:
from glob import glob
singlish = sorted(glob('/home/ubuntu/wav2vec2/singlish-test/*.wav'), key = lambda x: int(x.split('/')[-1].replace('.wav', '')))
len(singlish)

3579

In [5]:
with open('/home/ubuntu/wav2vec2/singlish-test.json') as fopen:
    singlish_label = json.load(fopen)
    
len(singlish_label)

3579

In [6]:
from sklearn.utils import shuffle

audio = singlish
labels = singlish_label
audio, labels = shuffle(audio, labels)
test_set = list(zip(audio, labels))
test_set[:10]

[('/home/ubuntu/wav2vec2/singlish-test/669.wav',
  'for good reasons each distrusts the other'),
 ('/home/ubuntu/wav2vec2/singlish-test/2175.wav',
  'boeing denied the trade case was motivated by those talks'),
 ('/home/ubuntu/wav2vec2/singlish-test/39.wav',
  'he was elected after winning the mandate of the people'),
 ('/home/ubuntu/wav2vec2/singlish-test/2044.wav',
  'we watched the drama unfold before our eyes'),
 ('/home/ubuntu/wav2vec2/singlish-test/2073.wav',
  'the witch put a spell on the prince but it was the wrong one'),
 ('/home/ubuntu/wav2vec2/singlish-test/799.wav',
  'this kind of packaging can pose a danger to animals'),
 ('/home/ubuntu/wav2vec2/singlish-test/796.wav',
  'the couple went on an enriching adventure together'),
 ('/home/ubuntu/wav2vec2/singlish-test/622.wav',
  'he has crawled into them mirror and torch in hand to ensure that they are ready'),
 ('/home/ubuntu/wav2vec2/singlish-test/2114.wav',
  'the student skated onto the stage to receive his certificate')

In [7]:
import kenlm
from pyctcdecode.language_model import LanguageModel

In [8]:
lm = malaya_speech.stt.language_model(model = 'bahasa-manglish-combined')

In [9]:
kenlm_model = kenlm.Model(lm)
language_model = LanguageModel(kenlm_model, alpha = 0.01, beta = 0.5)

In [10]:
y, _ = malaya_speech.load(test_set[0][0])
y1, _ = malaya_speech.load(test_set[1][0])

In [11]:
model = malaya_speech.stt.deep_transducer(model = 'large-conformer-singlish')

Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

2022-06-12 11:21:56.477913: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-12 11:21:56.482067: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-12 11:21:56.482090: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-06-12 11:21:56.482095: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-06-12 11:21:56.482174: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-06-12 11:21:56.482207: I

In [12]:
%%time

model.beam_decoder_lm([y, y1], language_model)

CPU times: user 11.1 s, sys: 567 ms, total: 11.7 s
Wall time: 7.37 s


['for good reasons it distrust the other',
 'boeing denied the trade case was motivated by those talks']

In [13]:
def calculate_cer(actual, hyp):
    """
    Calculate CER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    actual = actual.replace(' ', '')
    hyp = hyp.replace(' ', '')
    return Lev.distance(actual, hyp) / len(actual)


def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [14]:
from tqdm import tqdm

wer, cer = [], []
wer_lm, cer_lm = [], []

batch_size = 4
for i in tqdm(range(0, len(audio), batch_size)):
    
    batch_x = audio[i: i + batch_size]
    batch_y = labels[i: i + batch_size]
    
    ys = [malaya_speech.load(b)[0] for b in batch_x]
    pred = model.beam_decoder(ys)
    pred_lm = model.beam_decoder_lm(ys, language_model)
    
    for k in range(len(pred)):
        
        wer.append(calculate_wer(batch_y[k], pred[k]))
        cer.append(calculate_cer(batch_y[k], pred[k]))
        
        wer_lm.append(calculate_wer(batch_y[k], pred_lm[k]))
        cer_lm.append(calculate_cer(batch_y[k], pred_lm[k]))

  1%|          | 5/895 [00:47<2:18:12,  9.32s/it]2022-06-12 11:23:05.757435: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 35758080 exceeds 10% of free system memory.
2022-06-12 11:23:09.072205: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 35758080 exceeds 10% of free system memory.
  1%|          | 7/895 [01:09<2:31:20, 10.23s/it]2022-06-12 11:23:27.789107: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 33454080 exceeds 10% of free system memory.
2022-06-12 11:23:31.372088: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 33454080 exceeds 10% of free system memory.
100%|██████████| 895/895 [2:23:18<00:00,  9.61s/it]  


In [15]:
import numpy as np

np.mean(wer), np.mean(cer), np.mean(wer_lm), np.mean(cer_lm)

(0.070147337953737,
 0.035872015761097734,
 0.06981206558101838,
 0.03572307118016766)